In [1]:
import json
import pandas as pd
import numpy as np
import random
def dump_json(data, file_name):
    with open(f'./{file_name}.json', 'w') as fout:
        json.dump(data, fout, cls=DateTimeEncoder)
        
def read_json(file_name):
    with open(f'./{file_name}.json', 'r') as fout:
        return json.load(fout)

In [3]:
round_one = read_json('evidence_first_round')
round_two = read_json('evidence_second_round')
round_three = read_json('evidence_third_round')

In [5]:
tweet_dict = {}

def fill_dict(round_):
    misses = 0
    hits = 0
    max_answer_length_found = 0
    for i, hit in enumerate(round_):
        key = hit['tweet'] + hit['topic']
        
        max_answer_length_found = max(len(hit['answers']), max_answer_length_found)
        if key in tweet_dict:
            hits = hits + 1
            workers = {}
            for answ in [*tweet_dict[key]['answers'], *hit['answers']]:
                worker = answ['worker']
                if worker not in workers:
                    workers[worker] = answ
            max_answer_length_found = max(len(workers.values()), max_answer_length_found)
            tweet_dict[key]['answers'] = list(workers.values())
            
        else:
            tweet_dict[key] = hit.copy()
            misses = misses + 1
            
    print(misses, hits, max_answer_length_found)
            
fill_dict(round_one)
fill_dict(round_two)
fill_dict(round_three)
results = list(tweet_dict.values())

819 0 5
0 647 5
0 354 5


In [6]:
len(results)

819

In [7]:
dist = {}
for item in results:
    key = len(item['answers'])
    if key in dist:
        dist[key] = dist[key] + 1
    else:
        dist[key] = 1
        
dist

{5: 705, 4: 112, 3: 2}

In [17]:
results[0]['answers'][0]['answer']

'study'

In [73]:
evidence = []
for item in results:
    answers = item['answers']
    tweet = item['tweet']
    topic = item['topic']
    type_dist = {}
    for answer in answers:
        key = answer['answer']
        if key in type_dist:
            type_dist[key] = type_dist[key] + 1
        else:
            type_dist[key] = 1

    evidence.append((tweet, topic, type_dist, len(answers)))
        

In [74]:
evidence[55]

("<MENTION> <MENTION> <MENTION> But no one of us is buying meat directly from the producers.\nSorry,but it's hard to believe that reducing the production wld hurt the consumers.\nIt wld hurt the supermarkets.\nTHEY are the ones who wld import.\nFor their full looking shelves.",
 'Vegan and vegetarian diets should be encouraged',
 {'normative': 2, 'no evidence': 2, 'anecdotal': 1},
 5)

In [75]:
#for (tweet, topic_dist, sup) in arguments:
#    row = df[df.tweet == tweet]
#    df.loc[df.tweet == tweet, 'topic'] = (arg_dist['argumentative'] if 'argumentative' in arg_dist else 0)/sum(arg_dist.values())
df = pd.read_csv('claims-large.csv')
#df = pd.DataFrame(df_rec).sample(frac=1).drop_duplicates(subset=['id'])
#df.evidence_type = [{}]*len(df)
#df.evidence_type = df.evidence_type.astype('object')

In [76]:
for (tweet, topic, type_dist, num_answ) in evidence:
    
    evidence_prob = 0
    for key in type_dist:
        type_dist[key] = type_dist[key]/num_answ
        if key != 'no evidence':
            evidence_prob = max(type_dist[key], evidence_prob)
    
    df.loc[(df.tweet == tweet) & (df.topic == topic), 'evidence'] = evidence_prob
    df.loc[(df.tweet == tweet) & (df.topic == topic), 'evidence_type'] = [type_dist]
    
df

,tweet,id,lang,score,evidence,claim,procon,arg_type,evidence_type,topic,argumentative,unrelated_prob
0,BREAKING :- 🌟 \n\n• #ValimaiTrailer 🎯 Hits 1️⃣...,1.511979e+18,en,0.000000,0.0,0.0,0.0,NaN,NaN,We should reduce the consumption of meat,0.2,0.0
1,BREAKING :- 🌟 \n\n• #ValimaiTrailer 🎯 Hits 1️⃣...,1.511979e+18,en,0.000000,0.0,0.0,0.0,NaN,NaN,Plant based food should be encouraged,0.2,0.0
2,BREAKING :- 🌟 \n\n• #ValimaiTrailer 🎯 Hits 1️⃣...,1.511979e+18,en,0.000000,0.0,0.0,0.0,NaN,NaN,Meat alternatives should be encouraged,0.2,0.0
3,BREAKING :- 🌟 \n\n• #ValimaiTrailer 🎯 Hits 1️⃣...,1.511979e+18,en,0.000000,0.0,0.0,0.0,NaN,NaN,Vegan and vegetarian diets should be encouraged,0.2,0.0
4,BREAKING :- 🌟 \n\n• #ValimaiTrailer 🎯 Hits 1️⃣...,1.511979e+18,en,0.000000,0.0,0.0,0.0,NaN,NaN,We should pursue policies that promote sustain...,0.2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2980,<MENTION> <MENTION> Plant based?,1.511921e+18,en,0.549501,0.0,0.0,0.0,NaN,NaN,We should reduce the consumption of meat,0.2,0.0
2981,<MENTION> <MENTION> Plant based?,1.511921e+18,en,0.549501,0.0,0.0,0.0,NaN,NaN,Plant based food should be encouraged,0.2,0.0
2982,<MENTION> <MENTION> Plant based?,1.511921e+18,en,0.549501,0.0,0.0,0.0,NaN,NaN,Meat alternatives should be encouraged,0.2,0.0
2983,<MENTION> <MENTION> Plant based?,1.511921e+18,en,0.549501,0.0,0.0,0.0,NaN,NaN,Vegan and vegetarian diets should be encouraged,0.2,0.0


In [77]:
df[df.evidence > .5]

,tweet,id,lang,score,evidence,claim,procon,arg_type,evidence_type,topic,argumentative,unrelated_prob
23,<MENTION> <MENTION> This is so disappointing f...,1.511728e+18,en,0.663518,0.75,0.666667,0.0,NaN,"{'normative': 0.75, 'fact': 0.25}",Vegan and vegetarian diets should be encouraged,0.80,0.0
49,"<MENTION> <MENTION> Soil \nit holds water, it ...",1.511199e+18,en,0.000000,0.60,0.600000,0.0,NaN,"{'fact': 0.6, 'normative': 0.2, 'anecdotal': 0.2}",We should pursue policies that promote sustain...,0.75,0.4
51,<MENTION> <MENTION> <MENTION> When top athlete...,1.511461e+18,en,0.650733,0.60,0.400000,0.0,NaN,"{'fact': 0.6, 'expert': 0.2, 'anecdotal': 0.2}",Plant based food should be encouraged,0.60,0.0
58,<MENTION> <MENTION> Shit is crazy to me lol I ...,1.511392e+18,en,0.728965,1.00,0.400000,0.0,NaN,{'anecdotal': 1.0},Vegan and vegetarian diets should be encouraged,0.80,0.4
59,<MENTION> <MENTION> Shit is crazy to me lol I ...,1.511392e+18,en,0.728965,0.60,0.200000,0.0,NaN,"{'anecdotal': 0.6, 'fact': 0.2, 'normative': 0.2}",We should pursue policies that promote sustain...,0.80,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...
2952,<MENTION> This generation you speak of will ne...,1.510745e+18,en,0.632304,1.00,1.000000,-1.0,unrelated,{'normative': 1.0},Meat alternatives should be encouraged,0.80,1.0
2954,<MENTION> This generation you speak of will ne...,1.510745e+18,en,0.632304,0.60,0.666667,-1.0,NaN,"{'normative': 0.6, 'fact': 0.2, 'anecdotal': 0.2}",We should pursue policies that promote sustain...,0.80,0.0
2968,<MENTION> <MENTION> <MENTION> <MENTION> <MENTI...,1.511744e+18,en,0.000000,0.60,0.600000,0.0,NaN,"{'normative': 0.6, 'no evidence': 0.2, 'study'...",Vegan and vegetarian diets should be encouraged,0.80,0.4
2970,<MENTION> <MENTION> I eat meat all the time. I...,1.511875e+18,en,0.659221,0.60,0.200000,0.0,NaN,"{'normative': 0.6, 'anecdotal': 0.4}",We should reduce the consumption of meat,0.60,0.2


In [78]:
df.to_csv('evidence.csv', index = False)